# Lab: Missing data

Our data can contain missing values or benifit from transformations.

## Missing values

I have removed some of the ratings data in the office_ratings_missing.csv.

In [ ]:
import pandas as pd
df = pd.read_csv('data/raw/office_ratings_missing.csv', encoding = 'UTF-8')

In [ ]:
df.info()

We are missing values in our imdb_rating and total votes columns.

In [ ]:
df.shape[0] - df.count()

What to do? A quick solution is to either 0 the values or give them a roughtly central value (the mean).

To do this we use the fillna method.

In [ ]:
df['imdb_rating_with_0'] = df['imdb_rating'].fillna(0)

To fill with the mean.

In [ ]:
df['imdb_rating_with_mean'] = df['imdb_rating'].fillna(df['imdb_rating'].mean())

In [ ]:
df.head()

We can plot these to see what looks most reasonable (you can probably also make an educated guess here).

In [ ]:
df['imdb_rating_with_mean'].plot()

In [ ]:
df['imdb_rating_with_0'].plot()

Going with the mean seems quite sensible in this case. Especially as the data is guassian so the mean is probably an accurate represenation of the central value.

In [ ]:
ax = df['imdb_rating'].hist()
ax.axvline(df['imdb_rating'].mean(), color='k', linestyle='--')

## Transformations

Some statistical models, such as standard linear regression, require the predicted variable to be gaussian distributed (a single central point and a roughly symmetrical decrease in frequency, see [this Wolfram alpha page](https://www.wolframalpha.com/input/?i=gaussian+0+1).

The distribution of votes is positively skewed (most values are low).

In [ ]:
df['total_votes'].hist()

A log transformation can make this data closer to a guassian distributed data variable. For the log transformation we are going to use numpy (numerical python) which is a rather excellent library.

In [ ]:
import numpy as np

df['total_votes_log'] = np.log2(df['total_votes'])

In [ ]:
df['total_votes_log'].hist()

That is less skewed, but not ideal. Perhaps a square root transformation instead?

In [ ]:
df['total_votes_sqrt'] = np.sqrt(df['total_votes'])
df['total_votes_sqrt'].hist()

...well, maybe a inverse/reciprocal transformation. It is possible we have hit the limit on what we can do.

In [ ]:
df['total_votes_recip'] = np.reciprocal(df['total_votes'])
df['total_votes_recip'].hist()

At this point, I think we should conceded that we can make the distribution less positively skewed. However, transformation are not magic and we cannot turn a heavily positively skewed distribution into a normally distributed one.

Oh well.

We can calculate z scores though so we can plot both total_votes and imdb_ratings on a single plot. Currently, the IMDB scores vary between 0 and 10 whereas the number of votes number in the thousands.

In [ ]:
df['total_votes_z'] = (df['total_votes'] - df['total_votes'].mean()) / df['total_votes'].std()
df['imdb_rating_z'] = (df['imdb_rating'] - df['imdb_rating'].mean()) / df['imdb_rating'].std()

In [ ]:
df['total_votes_z'].hist()

In [ ]:
df['imdb_rating_z'].hist()

Now we can compare the trends in score and number of votes on a single plot.

We are going to use a slightly different approach to creating the plots. Called to the plot() method from Pandas actually use a library called matplotlib. We are going to use the pyplot module of matplotlib directly.

In [ ]:
import matplotlib.pyplot as plt

Convert the air date into a datetime object.

In [ ]:
df['air_date'] =  pd.to_datetime(df['air_date'])

Then call the subplots function fom pyplot to create two plots. From this we take the two plot axis (ax1, ax2) and call the method scatter for each to plot imdb_rating_z and total_votes_z.

In [ ]:
plt.style.use('ggplot')

f, (ax1, ax2) = plt.subplots(1, 2, sharey=True)
ax1.scatter( df['air_date'], df['imdb_rating_z'], color = 'red')
ax1.set_title('IMDB rating')
ax2.scatter( df['air_date'], df['total_votes_z'], color = 'blue')
ax2.set_title('Total votes')

In [ ]:
f

We can do better than that.

In [ ]:
plt.scatter(df['air_date'], df['imdb_rating_z'], color = 'red', alpha = 0.1)
plt.scatter(df['air_date'], df['total_votes_z'], color = 'blue', alpha = 0.1)

In [ ]:
?plt.scatter

We have done a lot so far. Exploring data in part 1, plotting data with the inbuilt Pandas methods in part 2 and dealing with both missing data and transfromations in part 3.

In part 4, we will look at creating your own functions, a plotting library called seaborn and introduce a larger dataset.